In [1]:
import torch
import torchaudio.datasets as datasets

In [2]:
dataset = datasets.LJSPEECH("../data")

In [3]:
from torch.utils.data import DataLoader

In [4]:
dataloader = DataLoader(dataset, batch_size=64)

In [59]:
from torch.nn.functional import pad
from torchaudio.functional import resample
from preprocess import quantize_and_onehot_waveform


def collate_fn(batched_data, resample_rate=0):
    if resample_rate > 0:
        resampled_waveforms = list(map(lambda x: resample(x[0], x[1], resample_rate), batched_data))
        batched_data = [(resampled_waveforms[i], resample_rate, batched_data[i][2], batched_data[i][3]) 
                        for i in range(len(batched_data))]
    
    max_size = max(list(map(lambda x: x[0].size(-1), batched_data)))
    padded_waveforms = list(map(lambda x: pad(x[0], (0, max_size-x[0].size(-1))), batched_data))
    quantized_waveforms = list(map(lambda x: quantize_and_onehot_waveform(x), padded_waveforms))
    stacked_waveforms = torch.stack(quantized_waveforms)
    
    normalized_transcripts = list(map(lambda x: x[3], batched_data))

    return (stacked_waveforms, normalized_transcripts)

In [60]:
from functools import partial

collate_fn_16000 = partial(collate_fn, resample_rate=16000)

In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else None)

In [62]:
from model import WaveNet

res_layers = 5
blocks = 2
casual_layers = 2

model = WaveNet(res_layers, blocks, casual_layers)

In [63]:
dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn_16000, num_workers=4)

In [64]:
for inputs, transcript in dataloader: break

AssertionError: Caught AssertionError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/ahad/miniconda3/envs/torch/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/ahad/miniconda3/envs/torch/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 61, in fetch
    return self.collate_fn(data)
  File "/tmp/ipykernel_11791/2408357070.py", line 14, in collate_fn
    quantized_waveforms = list(map(lambda x: quantize_and_onehot_waveform(x), padded_waveforms))
  File "/tmp/ipykernel_11791/2408357070.py", line 14, in <lambda>
    quantized_waveforms = list(map(lambda x: quantize_and_onehot_waveform(x), padded_waveforms))
  File "/home/ahad/programming/speech/wavenet/preprocess.py", line 5, in quantize_and_onehot_waveform
    assert inputs.dim() == 3 # To get correct outputs
AssertionError


In [27]:
inputs = inputs.to(device)
model = model.to(device)

In [34]:
receptive_field = (3-1) * sum([2**i for i in range(5)])*blocks + 1

In [32]:
model(inputs[:,:,:receptive_field + 2]).shape

torch.Size([1, 256, 1])

In [55]:
import torch.nn as nn
from preprocess import quantize_and_onehot_waveform

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCELoss()

In [57]:
for i in range(receptive_field+2, inputs.size(-1)):
    clipped_inputs = inputs[:,:,:i]
    target = quantize_and_onehot_waveform(inputs[:,:,i].view(1,1,-1))
    
    optimizer.zero_grad()
    pred = model(clipped_inputs)
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()


KeyboardInterrupt: 

In [ ]:
for inputs, transcript in dataloader:
    

In [13]:
import gc
torch.cuda.empty_cache()
gc.collect()

19